# Training a ConvNet PyTorch

In this notebook, you'll learn how to use the powerful PyTorch framework to specify a conv net architecture and train it on the CIFAR-10 dataset.

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.autograd import Variable
from torch.utils.data import DataLoader
from torch.utils.data import sampler
import importlib

import torchvision.datasets as dset
import torchvision.transforms as T
import torchvision.models as models

import numpy as np

import timeit
import customDataset
from customDataset import CustomDataset
from importlib import reload

In [2]:
reload(customDataset)

class ChunkSampler(sampler.Sampler):
    """Samples elements sequentially from some offset. 
    Arguments:
        num_samples: # of desired datapoints
        start: offset where we should start selecting from
    """
    def __init__(self, num_samples, start = 0):
        self.num_samples = num_samples
        self.start = start

    def __iter__(self):
        return iter(range(self.start, self.start + self.num_samples))

    def __len__(self):
        return self.num_samples
    
synth_dataset = CustomDataset()

-----Synth 2 Loaded------
-----Synth 3 Loaded------


In [3]:
train_loader = torch.utils.data.DataLoader(dataset=synth_dataset,
                                           batch_size=10, 
                                           shuffle=True)

In [4]:
dtype = torch.FloatTensor # the CPU datatype

# Constant to control how frequently we print train loss
print_every = 100

# This is a little utility that we'll use to reset the model
# if we want to re-initialize all our parameters
def reset(m):
    if hasattr(m, 'reset_parameters'):
        m.reset_parameters()

In [5]:
class Flatten(nn.Module):
    def forward(self, x):
        N, C, H, W = x.size() # read in N, C, H, W
        return x.view(N, -1)  # "flatten" the C * H * W values into a single vector per image

In [6]:
def train(model, loss_fn, optimizer, num_epochs = 1):
    for epoch in range(num_epochs):
        print('Starting epoch %d / %d' % (epoch + 1, num_epochs))
        model.train()
        for t, (x, y) in enumerate(train_loader):
            x_var = Variable(x.type(gpu_dtype))
            y_var = Variable(y.type(gpu_dtype).long())
            
            scores = model(x_var)
            
#             print("Iteration ", t)
#             print("scores=", scores.shape)
#             print(scores)
#             print("y_var=",y_var.shape)
#             print(y_var)
#             print("_____________________________________")
            
            loss = loss_fn(scores, y_var)
            
            if (t + 1) % print_every == 0:
                print('t = %d, loss = %.4f' % (t + 1, loss.item()))

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

def check_accuracy(model, loader):
#     if loader.dataset.train:
#         print('Checking accuracy on validation set')
#     else:
#         print('Checking accuracy on test set')   
    num_correct = 0
    num_samples = 0
    model.eval() # Put the model in test mode (the opposite of model.train(), essentially)
    for x, y in loader:
        x_var = Variable(x.type(gpu_dtype), volatile=True)

        scores = model(x_var)
        _, preds = scores.data.cpu().max(1)
        num_correct += (preds == y).sum()
        num_samples += preds.size(0)
    acc = float(num_correct) / num_samples
    print('Got %d / %d correct (%.2f)' % (num_correct, num_samples, 100 * acc))

In [7]:
torch.cuda.empty_cache()
gpu_dtype = torch.cuda.FloatTensor

# Train your model here, and make sure the output of this cell is the accuracy of your best model on the 
# train, val, and test sets. Here's some code to get you started. The output of this cell should be the training
# and validation accuracy on your best model (measured by validation accuracy).

# TODO: Tweak architecture

fixed_model_gpu = nn.Sequential(
        nn.Conv2d(in_channels=3,out_channels=32,kernel_size=7,stride=1),
        # 32x126x126
        nn.ReLU(inplace=True),
        nn.BatchNorm2d(num_features=32),
        #################################################################
        nn.Conv2d(in_channels=32,out_channels=32,kernel_size=7,stride=1),
        # 32x124x124
        nn.ReLU(inplace=True),
        nn.BatchNorm2d(num_features=32),
        nn.MaxPool2d(kernel_size=2,stride=2),
        # 32x62x62
        #################################################################
        nn.Conv2d(in_channels=32,out_channels=64,kernel_size=7,stride=1),
        # 64x60x60
        nn.ReLU(inplace=True),
        nn.BatchNorm2d(num_features=64),
        #################################################################
        nn.Conv2d(in_channels=64,out_channels=64,kernel_size=7,stride=1),
        # 64x58x58
        nn.ReLU(inplace=True),
        nn.BatchNorm2d(num_features=64),
        nn.MaxPool2d(kernel_size=2,stride=2),
        # 64x29x29
    
        Flatten(), # see above for explanation
        nn.Linear(33856, 4096), # affine layer
        nn.ReLU(inplace=True),
        nn.BatchNorm1d(4096),
#         nn.Linear(32768, 16384), # affine layer
#         nn.ReLU(inplace=True),
#         nn.BatchNorm1d(16384),
#         nn.Linear(16384, 8192),
#         nn.ReLU(inplace=True),
#         nn.BatchNorm1d(8192),
#         nn.Linear(8192, 4096),
#         nn.ReLU(inplace=True),
#         nn.BatchNorm1d(4096),
        nn.Linear(4096, 2048),
        nn.ReLU(inplace=True),
        nn.BatchNorm1d(2048),
        nn.Linear(2048, 1024),
        nn.ReLU(inplace=True),
        nn.BatchNorm1d(1024),
        nn.Linear(1024, 6),
        nn.ReLU(inplace=True),
        nn.BatchNorm1d(6)
      ).type(gpu_dtype)

model = fixed_model_gpu
loss_fn = torch.nn.CrossEntropyLoss().type(gpu_dtype)
optimizer = torch.optim.Adam(fixed_model_gpu.parameters(),lr = 1e-3)

train(model, loss_fn, optimizer, num_epochs=1)
check_accuracy(model, train_loader)

Starting epoch 1 / 1


ValueError: Expected more than 1 value per channel when training, got input size torch.Size([1, 4096])

In [ ]:
# model.eval()

In [ ]:
# model = fixed_model
# loss_fn = torch.nn.CrossEntropyLoss()
# optimizer = torch.optim.RMSprop(fixed_model_gpu.parameters(),lr = 1e-3)

# train(model, loss_fn, optimizer, num_epochs=3)
# check_accuracy(model, loader_val)

In [ ]:
# best_model = model
# check_accuracy(best_model, loader_test)